In [3]:
import basefunc
import numpy as np
import os

In [4]:
#Safaty#

def f_normal(inputs):
    """
    According to lower design standards assumption (see supplementary Table S1)
    
                design return period/years: motorway et.al, tertiary, railways
    high income：100, 30
    upper middle income：50, 10
    low middle/low income：10, 2
    

    inputs[0]: income group index. 1: low income group, 2: lower middle income group, 3: upper middle income group, 4: high income group
    inputs[1]: assets_type. 1: MV,  2: LV
    inputs[-18:-9]: new RX1D under design standards
    inputs[-9:]: historical RX1D under design standards
                 (0: 1.1years, 1: 2years, 2: 5years, 3: 10years, 4: 20years, 5: 30years, 6: 50years, 7: 100years, 8: 200year)
                 
    output: the proportion of new return period to the historical design return period
    """

    income=inputs[0]
    assets_type=inputs[1]
    new_pre=inputs[-18:-9]
    his_pre=inputs[-9:]
    y=-1.0
    
    if assets_type==1:
        if income==1 or income==2: 
            y=new_pre[3]/his_pre[3]                                
        elif income==3: #upper middle
            y=new_pre[6]/his_pre[6]
        elif  income==4: 
            y=new_pre[7]/his_pre[7]  
        else:
            y=-1.0   
    elif assets_type==2:
        if income==1 or income==2:
             y=new_pre[1]/his_pre[1]
        elif income==3:
            y=new_pre[3]/his_pre[3]             
        elif income==4:    
            y=new_pre[5]/his_pre[5]             
        else:
            y=-1.0
       
    return  y



# main
types=['MV','LV']


his_pre=basefunc.getRaster('./results_for_gev/historical_mme.tif')

#by types
types=['MV','LV']
SCENARIO_YEARS = {
    
    'ssp126': [(2030, 2060), (2070, 2100)],
    'ssp245': [(2030, 2060), (2070, 2100)],
    'ssp370': [(2030, 2060), (2070, 2100)],
    'ssp585': [(2030, 2060), (2070, 2100)],
}

ref_tif='./data/ref_tif.tif'

for name in types:
    for scen, ranges in SCENARIO_YEARS.items():
        for time_low,time_up in ranges:
            if name=='MV':
                assets_type=1.0
            if name=='LV':
                assets_type=2.0
            
            assets_type=np.full((1,600,1440),assets_type)
            income=basefunc.getRaster("D:/QQ/PhD/cmip6/income_group.tif")[:600,:]
            income=income.reshape(1,600,1440)
            new_pre=basefunc.getRaster('./results_for_gev/'+scen+'pre'+str(time_low)+str(time_up)+'_mme.tif')

            array=np.concatenate((income,assets_type,new_pre,his_pre),axis=0)
            amf=np.apply_along_axis(f_normal,0,array)
            
            output_path='./results_for_amf/f_normal/'+name+'_under_'+scen+'_'+str(time_low)+str(time_up)+'_mme.tif'
            basefunc.array2Raster(amf,ref_tif,output_path)

    
# mean results of all types 
for scen, ranges in SCENARIO_YEARS.items():
    for time_low,time_up in ranges:
        data=np.full((6,600,1440),np.nan)
        i=0
        for name in types:
            data[i,:,:]=basefunc.getRaster('./results_for_amf/f_normal/'+name+'_under_'+scen+'_'+str(time_low)+str(time_up)+'_mme.tif')
            i=i+1
        data[data==-1]=np.nan
        data[data<0]=np.nan  # filter the parts that did not pass ks test
        amf_mean=np.nanmean(data,axis=0)
         

        output_path='./results_for_amf/f_normal/amf_under_'+scen+'_'+str(time_low)+str(time_up)+'_mme.tif'
        basefunc.array2Raster(amf_mean,ref_tif,output_path)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_12264\664831438.py:141: RuntimeWarning: Mean of empty slice
  amf_mean=np.nanmean(data,axis=0)
